<a href="https://colab.research.google.com/github/Ting-Yi-Wang/AI_homework/blob/main/HW5_tenkey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW5 RAG

## 1. 請完成以下程式 (50%)

以下程式碼有許多不完整的地方，請依照原本的程式框架，補齊所有未完成的部分，請確保所有程式碼能完整執行並得到合理的回答結果。

In [3]:
# ✅ STEP 1: 安裝需要的套件

!pip install -q sentence-transformers faiss-cpu requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.3 MB/s eta 0:00:00


In [4]:
# ✅ STEP 2: 載入必要模組

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import requests
import pandas as pd

In [5]:
### 非台大學生請執行以下程式碼獲取語料 ###

!gdown 1AF2g2WTtXQwb02S5EAD7aL48c9BkV1ed

### 非台大學生請執行以上程式碼獲取語料 ###

Downloading...
From: https://drive.google.com/uc?id=1AF2g2WTtXQwb02S5EAD7aL48c9BkV1ed
To: /content/dcard-top100.csv
100% 186k/186k [00:00<00:00, 92.4MB/s]


In [6]:
# ✅ STEP 3: 準備你的語料

import pandas as pd # Make sure pandas is imported
docs_df = pd.read_csv("dcard-top100.csv")

documents = docs_df["content"].tolist()
docs_df

,ID,title,content
0,235443715,三更 21歲這年我差點被活活燒死,部分回應在B117 \r\n謝謝各位的留言，我都有看完\r\n好的不好的，我都接受謝謝大家🙇...
1,235442609,超狂學經歷！195公分帥家教徵學生,https://i.imgur.com/REIEzSd.jpg\r\n\r\n身高195公分...
2,235441998,我的模特界時間管理大師前男友,看過這麼多在Dcard、PTT上的感情渣事和創作文\r\n從沒想過如此荒謬像八點檔的事情居然...
3,235441259,豆皮加爆,剛剛吃小火鍋，跟店員說不要金針菇（怕卡牙縫），於是店員幫我換其他配料..…\r\n\r\n沒...
4,235442693,這樣女生該追嗎,已經約好見面，到了當天晚上七點半才回，我是被耍了嗎 \r\n如下圖\r\n\r\n\r\nh...
...,...,...,...
95,235444562,主題：💟愛情塔羅💟 他要的感情關係vs 我要的感情關係為何？兩個人的近期發展（曖昧/交往中/...,🤗感謝我們之間的連結帶領你來到這裡！ \r\n\r\n主題：💟愛情塔羅💟 他要的感情關係vs...
96,235441027,#分享 老花古董包 LV 方盒子｜Celine 化妝箱,近期購入的兩款古董包 可能因為是新歡 所以都讓我愛不釋手(๑•̀ •́)و✧\r\n不過正是...
97,235447320,在公車上遇到的超尷尬事件,剛剛在公車上有一個阿伯下車前跟我說：妹妹你很有愛心欸 讓位置給別人 不像有些人都在裝睡\r\...
98,235440749,#大眾占卜 🥺。他現在對我的想法是?他的下一步是什麼?我能為這段關係做什麼樣的努力❣,大家好這裡是金魚🙏\r\n這次的占卜適合斷聯、曖昧、復合的族群\r\n\r\n再上圖之前先置...


In [7]:
# ✅ STEP 4: 建立 embedding 模型與 FAISS 索引

embedding_model = SentenceTransformer("shibing624/text2vec-base-multilingual")

# 轉換為向量並正規化（便於 cosine 相似度）
doc_embeddings = embedding_model.encode(documents, normalize_embeddings=True)

# 建立向量索引
index = faiss.IndexFlatIP(doc_embeddings.shape[1])
index.add(doc_embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/88.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
#### 作業完成後請刪除此 code chunk 後再上傳作業 ####

# 你的 api token

HUGGINGFACE_TOKEN = "***********"

#### 作業完成後請刪除此 code chunk 後再上傳作業 ####

In [17]:
# 🤖 STEP 5: 呼叫 HuggingFace 模型 API

import requests

API_URL = "https://api-inference.huggingface.co/models/ClueAI/ChatYuan-large-v2"

headers = {"Authorization": f"Bearer {HUGGINGFACE_TOKEN}"}

# 請完成 prompt
def generate_answer(query, context_docs):
    context = "\n".join(f"- {doc[:300]}" for doc in context_docs)  # 避免太長
    prompt = f"""你是一個，請根據以下資料回答問題，若沒有相關資訊請回答「找不到答案」。

資料：{context}

問題：{query}

請用一到兩句話回答。
"""
    payload = {"inputs": prompt}

    try:
        response = requests.post(API_URL, headers=headers, json=payload, timeout=60)
        response.raise_for_status()
        data = response.json()

        # 根據 API 回傳格式解析
        if isinstance(data, list) and "generated_text" in data[0]:
            return data[0]["generated_text"]
        else:
            return f"⚠️ 無法解析回應：{data}"

    except requests.exceptions.RequestException as e:
        return f"❌ API 請求錯誤：{e}"
    except Exception as e:
        return f"❌ 發生錯誤：{e}"



In [18]:
def search(query, top_k=3):

  query_vec = embedding_model.encode([query], normalize_embeddings=True)
  scores, indices = index.search(query_vec, top_k)
  matched_docs = [documents[i] for i in indices[0]]

  return matched_docs


In [26]:
# ✅ STEP 6: 使用者輸入問題，找出最相關的內容

query = "去哪邊吃晚餐"

relevant_docs = search(query, top_k=3)
answer = generate_answer(query, relevant_docs)

print("🔎 相關段落：\n", "\n---\n".join(relevant_docs)[:100])
print("\n💬 AI 回答：\n", answer)

🔎 相關段落：
 以下為實際案例：

1.跟我約好了時間 卻可以因為媽媽叫他做什麼事而遲到或是毀約
他快要生日的時候 我提前很多天跟他說 你生日當天要留給我喔 我要幫你慶生 到了前幾天他告訴我他要陪媽媽吃完中餐才

💬 AI 回答：
 答案： 去臺灣的臺灣灣 謝謝!


## 2. 請回答以下問題 (50%)

請嘗試輸入至少 3 個問題給你的問答系統，並觀察模型回答的準確性與表現：

- 你問了哪些問題？

1.哪邊可以採草莓:
 相關段落：
 🔅來來牛奶草莓園🍓🍓
一轉進水尾坪這裡就可以看到好多草莓園
但是這次很堅持的是要去來來牛奶草莓園
主要的是看重這家草莓園還有提供草莓斗笠
可以讓你一邊戴著可愛斗笠一邊採收草莓
好像自己就是個

💬 AI 回答：
 答案： 水尾坪 水尾坪是台灣最大的草莓，

2.生日有好的建議嗎?
相關段落：
 以下為實際案例：
跟我約好了時間 卻可以因為媽媽叫他做什麼事而遲到或是毀約
他快要生日的時候 我提前很多天跟他說 你生日當天要留給我喔 我要幫你慶生 到了前幾天他告訴我他要陪媽媽吃完中餐才

💬 AI 回答：
 這個問題問的很， 讓我們無法回答。 請

3. 去哪邊吃晚餐?
相關段落：
 以下為實際案例：
跟我約好了時間 卻可以因為媽媽叫他做什麼事而遲到或是毀約
他快要生日的時候 我提前很多天跟他說 你生日當天要留給我喔 我要幫你慶生 到了前幾天他告訴我他要陪媽媽吃完中餐才

💬 AI 回答：
 答案： 去臺灣的臺灣灣 謝謝!

- 模型是否成功從語料中找到與問題相關的內容？
除了第一題有找到，第二題找到的資料有點關係，第三題找到的完全不相關

- 回答是否合理？是否有亂編的情況？
除第一題外，其他的回答不合理，第三題完全亂編

- 若問題無法回答，模型是否能正確回應「找不到答案」？
第二題，模型無法回答，也無法回應「找不到答案」

請根據你的觀察，簡述模型整體表現的優缺點。

優點：
能擷取部分相關資訊： 至少在某些情況下（第一個問題），模型能夠從語料庫中找到與問題相關的內容，並提取出部分關鍵資訊。

缺點：
準確度不足： 模型在許多情況下無法找到真正相關的內容，或提取出錯誤的資訊，導致回答不準確或不完整。
容易亂編： 當模型無法找到相關資訊時，有時會傾向於編造答案，而不是誠實地回答「找不到答案」
回應不穩定： 模型的表現不夠穩定，有時能找到相關資訊，有時卻完全找不到，或給出不合理的回答。
無法處理複雜問題： 對於較為複雜或抽象的問題，模型的理解能力明顯不足，無法給出令人滿意的答案。
找不到答案時，無法正確回應： 當模型無法回答問題時，並不能總是正確地回應「找不到答案」。

整體而言，這個模型的表現還有很大的改進空間。 雖然它具備基本的語義理解和搜尋能力，但在準確度、穩定性和泛化能力方面仍有不足。


